In [16]:
# Import dataframes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [17]:
# Load Dataset

df = pd.read_csv("airbnb_dataset.csv")
pd.set_option('display.max_columns', None)  # Show all columns, no truncation
df.head()

C:\Users\hmsha\AppData\Local\Temp\ipykernel_2088\2361970955.py:3: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("airbnb_dataset.csv")


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,country code,instant_bookable,cancellation_policy,room type,Construction year,price,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,US,False,strict,Private room,2020.0,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,US,False,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,US,True,flexible,Private room,2005.0,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,US,True,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,US,False,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


** Step 1: Clean and pre-process data **

In [18]:
# Find duplicates
df.duplicated().sum()

np.int64(541)

In [19]:
# Drop duplicates
df.drop_duplicates(inplace=True)
df.duplicated().sum()

np.int64(0)

In [20]:
# Identifying non-informative features to downstream analysis
df.nunique()


id                                102058
NAME                               61281
host id                           102057
host_identity_verified                 2
host name                          13190
neighbourhood group                    7
neighbourhood                        224
lat                                21991
long                               17774
country                                1
country code                           1
instant_bookable                       2
cancellation_policy                    3
room type                              4
Construction year                     20
price                               1151
service fee                          231
minimum nights                       153
number of reviews                    476
last review                         2477
reviews per month                   1016
review rate number                     5
calculated host listings count        78
availability 365                     438
house_rules     

Above cell Indicates that columns country, country code, and license have only 1 unique value, which means all of them are same. So we can drop them to streamline analysis.

In [21]:
# Dropping non-informative columns.
df.drop(["country", "country code", "license"], axis=1, inplace=True) 


In [22]:
# convert last_review column datatype to datetime format
df["last review"] = pd.to_datetime(df['last review'], format="%m/%d/%y", exact=False)
df["last review"].head(5)

0   2020-10-19
1   2020-05-21
2          NaT
3   2020-07-05
4   2020-11-19
Name: last review, dtype: datetime64[ns]

In [23]:
# convert price and service fee columns into floats, and remove non-numerical characters such as $ or ,
df["price"] = df["price"].str.replace(r"[$,]", "", regex=True).astype(float)
df["service fee"] = df["service fee"].str.replace(r"[$,0]", "", regex=True).astype(float)

In [24]:
# Get sum of null values in each column
df.isna().sum()

id                                    0
NAME                                250
host id                               0
host_identity_verified              289
host name                           404
neighbourhood group                  29
neighbourhood                        16
lat                                   8
long                                  8
instant_bookable                    105
cancellation_policy                  76
room type                             0
Construction year                   214
price                               247
service fee                         273
minimum nights                      400
number of reviews                   183
last review                       15832
reviews per month                 15818
review rate number                  319
calculated host listings count      319
availability 365                    448
house_rules                       51842
dtype: int64

In [25]:
# Drop rows with null values except the ones where only rows with eithier last review or 
df["reviews per month"].sort_values(ascending=False)

49653     90.00
49827     84.49
49654     65.74
74683     58.50
42076     58.50
          ...  
102035      NaN
102037      NaN
102045      NaN
102051      NaN
102056      NaN
Name: reviews per month, Length: 102058, dtype: float64

In [26]:
# Verify that all columns' datatype are correct or most useful.
df.dtypes

id                                         int64
NAME                                      object
host id                                    int64
host_identity_verified                    object
host name                                 object
neighbourhood group                       object
neighbourhood                             object
lat                                      float64
long                                     float64
instant_bookable                          object
cancellation_policy                       object
room type                                 object
Construction year                        float64
price                                    float64
service fee                              float64
minimum nights                           float64
number of reviews                        float64
last review                       datetime64[ns]
reviews per month                        float64
review rate number                       float64
calculated host list

In [27]:
# Drops all rows in which any columns other than the ones listed have nulls.
cols_to_ignore = ["last_review", "reviews per month", "house_rules"]
df.dropna(subset=[col for col in df if col not in cols_to_ignore], inplace=True)

In [28]:
# Verifying null counts.
df.isna().sum()

id                                    0
NAME                                  0
host id                               0
host_identity_verified                0
host name                             0
neighbourhood group                   0
neighbourhood                         0
lat                                   0
long                                  0
instant_bookable                      0
cancellation_policy                   0
room type                             0
Construction year                     0
price                                 0
service fee                           0
minimum nights                        0
number of reviews                     0
last review                           0
reviews per month                     6
review rate number                    0
calculated host listings count        0
availability 365                      0
house_rules                       44296
dtype: int64

In [29]:
# Save modified CSV for future analysis.
df.to_csv("cleaned_airbnb_dataset.csv", index=False)